# Import

In [1]:
import pandas as pd
import numpy as np
import folium
from folium.plugins import HeatMap
from IPython.display import HTML
import matplotlib.pyplot as plt
%matplotlib inline
import json

# Restaurant Inspection Scores

In [2]:
#Read in csv file with Restaurant Inspection Scores
scores = pd.read_csv('data/restaurant_inspec_scores.csv')

In [5]:
#For now, only use restaurants that have latitude/longitude
scores_lat = scores[scores['business_latitude'].notnull()]
scores_lat

,business_id,business_name,business_address,business_city,business_state,business_postal_code,business_latitude,business_longitude,business_location,business_phone_number,inspection_id,inspection_date,inspection_score,inspection_type,violation_id,violation_description,risk_category
22,3196,United Irish Cultural Center,2700 45th Ave,San Francisco,CA,94116,37.735943,-122.503086,POINT (-122.503086 37.735943),NaN,3196_20170201,2/1/2017 0:00,NaN,Reinspection/Followup,NaN,NaN,NaN
33,5827,HILLCREST ELEMENTARY SCHOOL,810 SILVER Ave,San Francisco,CA,94134,37.729016,-122.419253,POINT (-122.419253 37.729016),1.415546e+10,5827_20190903,9/3/2019 0:00,88.0,Routine - Unscheduled,5827_20190903_103120,Moderate risk food holding temperature,Moderate Risk
63,5959,ST. JOHN'S SNACK BAR,925 CHENERY St,San Francisco,CA,94131,37.735264,-122.437679,POINT (-122.437679 37.735264),1.415559e+10,5959_20180403,4/3/2018 0:00,100.0,Routine - Unscheduled,NaN,NaN,NaN
89,1231,Yamo Restaurant,3406 18th St,San Francisco,CA,94110,37.761882,-122.419599,POINT (-122.419599 37.761882),NaN,1231_20190904,9/4/2019 0:00,NaN,New Construction,NaN,NaN,NaN
114,68451,Buckhorn Grill,101 04th St #1060,San Francisco,CA,94103,37.784548,-122.404190,POINT (-122.40419 37.784548),NaN,68451_20180226,2/26/2018 0:00,74.0,Routine - Unscheduled,68451_20180226_103119,Inadequate and inaccessible handwashing facili...,Moderate Risk
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53850,7643,PACIFIC SUPER MARKET,2900 ALEMANY Blvd,San Francisco,CA,94112,37.712148,-122.452076,POINT (-122.452076 37.712148),1.415533e+10,7643_20161101,11/1/2016 0:00,NaN,Complaint,NaN,NaN,NaN
53851,60473,Dong Baek Korean Restaurant,631 O'Farrell St,San Francisco,CA,94109,37.785503,-122.415157,POINT (-122.415157 37.785503),NaN,60473_20190228,2/28/2019 0:00,79.0,Routine - Unscheduled,60473_20190228_103109,Unclean or unsanitary food contact surfaces,High Risk
53852,1843,Ma's Dim Sum & Cafe,1315 Powell St,San Francisco,CA,94133,37.796683,-122.410134,POINT (-122.410134 37.796683),NaN,1843_20171129,11/29/2017 0:00,NaN,Reinspection/Followup,NaN,NaN,NaN
53853,11365,LE PETITT'S KITCHEN,265 GOLDEN GATE AVE,San Francisco,CA,94109,37.781690,-122.414823,POINT (-122.414823 37.78169),NaN,11365_20170216,2/16/2017 0:00,96.0,Routine - Unscheduled,11365_20170216_103119,Inadequate and inaccessible handwashing facili...,Moderate Risk


# Airbnb Listings

In [3]:
#Read in csv file with Airbnb data
airbnb = pd.read_csv('data/sf_airbnb_listings.csv')

# SF Map with Airbnb HeatMap

In [81]:
sf_coord = [37.76, -122.45] #Coordination of city of San Francisco

#Set up basic map of San Francisco
sf_map = folium.Map(location=sf_coord,
                    tiles='cartodbpositron',
                    zoom_start=12.5)

In [14]:
#Add a heat map based on data in Airbnb table to the San Francisco map
HeatMap(data=airbnb[['latitude', 'longitude']].groupby(['latitude', 'longitude']).sum().reset_index().values.tolist(), radius=8, max_zoom=13).add_to(sf_map)
sf_map

# Add Inspec Risk Data to SF Map

In [15]:
#Color dictionary per risk factor
colordict = {'High Risk': 'red', 'Moderate Risk': 'yellow', 'Low Risk': 'green'}

#Add restaurant data with different colors based on the risk factor
for lat, lon, risk, name in zip(scores_lat['business_latitude'], scores_lat['business_longitude'], scores_lat['risk_category'], scores_lat['business_name']):
    if risk in ['High Risk', 'Moderate Risk', 'Low Risk']:
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            color='b',
            fill_color=colordict[risk],
            fill=True,
            fill_opacity=1
            ).add_to(sf_map)

In [16]:
#Save the map into a callable html file (This is how to save time and not cause an error in loading lots of data)
sf_map.save('sf_data_visualization.html')

In [18]:
#Import Folium interactive html file
HTML('<iframe src=sf_data_visualization.html width=1000 height=700></iframe>')

# Airbnb with \# of nearby restaurants

In [43]:
a_w_rest = pd.read_csv('airbnb_with_#of_restuarants.csv')

In [44]:
sf_coord = [37.76, -122.45] #Coordination of city of San Francisco

#Set up basic map of San Francisco
sf_map_2 = folium.Map(location=sf_coord,
                    tiles='cartodbpositron',
                    zoom_start=12.5)

In [45]:
for lat, lon, num in zip(a_w_rest['latitude'], a_w_rest['longitude'], a_w_rest['rest_num']):
    folium.Circle(
        [lat, lon],
        radius=num,
        color='b',
        fill_color='crimson',
        fill=True,
        ).add_to(sf_map_2)

In [7]:
sf_map_2.save('trying_this.html')

In [8]:
HTML('<iframe src=trying_this.html width=1000 height=700></iframe>')

/opt/conda/lib/python3.6/site-packages/IPython/core/display.py:689: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


# Processed data with score and location

In [4]:
df = pd.read_csv('data/processed.csv')

In [5]:
df = df[df['inspection_score'].notnull()]

# Categorize score based on SF data
- above 90: good
- 86-90: adequate
- 71-85: needs improvement
- <= 70: poor

In [54]:
def categorize_score (score):
    if (score > 90):
        return 'Good'
    elif (score >= 86) & (score <= 90):
        return 'Adequate'
    elif (score >= 71) & (score <= 85):
        return 'Needs improvement'
    elif (score <= 70):
        return 'Poor'

In [55]:
df['scores'] = df['inspection_score'].apply(categorize_score)

# SF Map with categorized score colors

In [56]:
sf_coord = [37.76, -122.45] #Coordination of city of San Francisco

#Set up basic map of San Francisco
sf_map_3 = folium.Map(location=sf_coord,
                    tiles='cartodbpositron',
                    zoom_start=12.5)

In [57]:
#Color dictionary per risk factor
colordict = {'Poor': 'red', 'Needs improvement': 'orange', 'Adequate': 'yellow', 'Good': 'green'}

#Add restaurant data with different colors based on the risk factor
for lat, lon, score in zip(df['business_latitude'], df['business_longitude'], df['scores']):
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        color='b',
        fill_color=colordict[score],
        fill=True,
        fill_opacity=1
        ).add_to(sf_map_3)

In [58]:
sf_map_3.save('inspect_score.html')

In [59]:
HTML('<iframe src=inspect_score.html width=1000 height=700></iframe>')

/opt/conda/lib/python3.6/site-packages/IPython/core/display.py:689: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


# SF Map with categorized score colors and Stamen Toner

In [60]:
sf_coord = [37.76, -122.45] #Coordination of city of San Francisco

#Set up basic map of San Francisco
sf_toner = folium.Map(location=sf_coord,
                    tiles='Stamen Toner',
                    zoom_start=12.5)

In [61]:
#Color dictionary per risk factor
colordict = {'Poor': 'red', 'Needs improvement': 'orange', 'Adequate': 'yellow', 'Good': 'green'}

#Add restaurant data with different colors based on the risk factor
for lat, lon, score in zip(df['business_latitude'], df['business_longitude'], df['scores']):
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        color='b',
        fill_color=colordict[score],
        fill=True,
        fill_opacity=1
        ).add_to(sf_toner)

In [82]:
HeatMap(data=airbnb[['latitude', 'longitude']].groupby(['latitude', 'longitude']).sum().reset_index().values.tolist(), radius=8, max_zoom=13).add_to(sf_toner)

In [83]:
sf_toner.save('inspect_score_toner.html')

In [84]:
HTML('<iframe src=inspect_score_toner.html width=1000 height=700></iframe>')

# Airbnb with only good restaurants

In [89]:
sf_coord = [37.76, -122.45] #Coordination of city of San Francisco

#Set up basic map of San Francisco
sf_toner_2 = folium.Map(location=sf_coord,
                    tiles='Stamen Toner',
                    zoom_start=12.5)

In [90]:
#Add restaurant data with different colors based on the risk factor
for lat, lon, score in zip(df['business_latitude'], df['business_longitude'], df['scores']):
    if score == 'Good':
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            color='b',
            fill_color='green',
            fill=True,
            fill_opacity=1
            ).add_to(sf_toner_2)

In [91]:
HeatMap(data=airbnb[['latitude', 'longitude']].groupby(['latitude', 'longitude']).sum().reset_index().values.tolist(), radius=8, max_zoom=13).add_to(sf_toner_2)

In [92]:
sf_toner_2.save('inspect_score_toner_good.html')

In [93]:
HTML('<iframe src=inspect_score_toner_good.html width=1000 height=700></iframe>')

In [3]:
listing = pd.read_csv('data/listings.csv')
listing           

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,958,"Bright, Modern Garden Unit - 1BR/1B",1169,Holly,NaN,Western Addition,37.76931,-122.43386,Entire home/apt,170,1,226,2019-11-17,1.79,1,71
1,3850,Charming room for two,4921,Kevin,NaN,Inner Sunset,37.75402,-122.45805,Private room,99,1,175,2019-12-01,1.38,3,74
2,5858,Creative Sanctuary,8904,Philip And Tania,NaN,Bernal Heights,37.74511,-122.42102,Entire home/apt,235,30,111,2017-08-06,0.86,1,1
3,7918,A Friendly Room - UCSF/USF - San Francisco,21994,Aaron,NaN,Haight Ashbury,37.76669,-122.45250,Private room,65,32,18,2019-09-10,0.14,9,365
4,8142,Friendly Room Apt. Style -UCSF/USF - San Franc...,21994,Aaron,NaN,Haight Ashbury,37.76487,-122.45183,Private room,65,32,8,2018-09-12,0.13,9,365
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8528,40544136,Entire apartment in Pacific Heights/beautiful ...,39425074,Sam,NaN,Pacific Heights,37.79017,-122.43869,Entire home/apt,95,30,0,NaN,NaN,2,337
8529,40546145,Huge Private Room; 8 min to Dwntwn SF & SFO,313682675,Connor,NaN,Bayview,37.71977,-122.39214,Private room,55,1,0,NaN,NaN,2,59
8530,40547706,Welcome onboard a beautiful Sailboat!,10994095,Aline,NaN,Treasure Island/YBI,37.81704,-122.36999,Entire home/apt,100,30,0,NaN,NaN,1,173
8531,40560328,Lg 1 Bed/1 Bath w/ Living & Dining Rm in Top Area,24087059,Simon,NaN,Pacific Heights,37.79179,-122.43021,Entire home/apt,295,2,0,NaN,NaN,2,4


# Choropleth Map based on average cost per neighbourhood

In [4]:
avg_listing = listing[['neighbourhood','price']].groupby('neighbourhood').mean().reset_index()

In [9]:
san_fran_geo = 'san-francisco.geojson'

In [28]:
sf_coord = [37.76, -122.45] #Coordination of city of San Francisco

#Set up basic map of San Francisco
sf_choro = folium.Map(location=sf_coord,
                    tiles='cartodbpositron',
                    zoom_start=12.5)

In [29]:
folium.Choropleth(
    geo_data=san_fran_geo,
    name='choropleth',
    data=avg_listing,
    columns=['neighbourhood', 'price'],
    key_on='feature.properties.name',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Average Airbnb Price ($)'
).add_to(sf_choro)

#Save to html
sf_choro.save('choropleth_sf.html')

In [30]:
HTML('<iframe src=choropleth_sf.html width=900 height=700></iframe>')